In [3]:
# Lab 9 XOR-back_prop
import numpy as np
import tensorflow as tf
assert(tf.__version__.find('2') == 0)
tf.random.set_seed(777)  # for reproducibility

learning_rate = 0.1

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.Variable(x_data)
Y = tf.Variable(y_data)

W1 = tf.Variable(tf.random.normal([2, 2]), name='weight1')
b1 = tf.Variable(tf.random.normal([2]), name='bias1')
l1 = lambda: tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random.normal([2, 1]), name='weight2')
b2 = tf.Variable(tf.random.normal([1]), name='bias2')
Y_pred = lambda: tf.sigmoid(tf.matmul(l1(), W2) + b2)

# cost/loss function
@tf.function
def cost():
    return -tf.reduce_mean(Y * tf.math.log(Y_pred()) + (1 - Y) *
                           tf.math.log(1 - Y_pred()))

# Network
#          p1     a1           l1     p2     a2           l2 (y_pred)
# X -> (*) -> (+) -> (sigmoid) -> (*) -> (+) -> (sigmoid) -> (loss)
#       ^      ^                   ^      ^
#       |      |                   |      |
#       W1     b1                  W2     b2

# Loss derivative
d_Y_pred = lambda: (Y_pred() - Y) / (Y_pred() * (1.0 - Y_pred()) + 1e-7)

# Layer 2
d_sigma2 = lambda: Y_pred() * (1 - Y_pred())
d_a2 = lambda: d_Y_pred() * d_sigma2()
d_p2 = d_a2
d_b2 = d_a2
d_W2 = lambda: tf.matmul(tf.transpose(l1()), d_p2())

# Mean
d_b2_mean = lambda: tf.reduce_mean(d_b2(), axis=[0])
d_W2_mean = lambda: d_W2() / tf.cast(tf.shape(l1())[0], dtype=tf.float32)

# Layer 1
d_l1 = lambda: tf.matmul(d_p2(), tf.transpose(W2))
d_sigma1 = lambda: l1() * (1 - l1())
d_a1 = lambda: d_l1() * d_sigma1()
d_b1 = d_a1
d_p1 = d_a1
d_W1 = lambda: tf.matmul(tf.transpose(X), d_a1())

# Mean
d_W1_mean = lambda: d_W1() / tf.cast(tf.shape(X)[0], dtype=tf.float32)
d_b1_mean = lambda: tf.reduce_mean(d_b1(), axis=[0])

# Weight update
step = [
  lambda: W2.assign(W2 - learning_rate * d_W2_mean()),
  lambda: b2.assign(b2 - learning_rate * d_b2_mean()),
  lambda: W1.assign(W1 - learning_rate * d_W1_mean()),
  lambda: b1.assign(b1 - learning_rate * d_b1_mean())
]

# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = lambda: tf.cast(Y_pred() > 0.5, dtype=tf.float32)
accuracy = lambda: tf.reduce_mean(tf.cast(tf.equal(predicted(), Y), dtype=tf.float32))

# Launch graph
# @tf.function
def run():
    for i in range(10001):
        step[0]()
        step[1]()
        step[2]()
        step[3]()
        cost_val = cost()
        if i % 1000 == 0:
            tf.print(
                "Step: ", i, "\t Loss: ", cost_val
            )
            tf.print(
                "W1: ", W1, "\t W2: ", W2
            )
            
print("Running...")
run()

# Accuracy report
h, c, a = Y_pred(), predicted(), accuracy()
print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

'''
Hypothesis:  [[ 0.01338224]
 [ 0.98166382]
 [ 0.98809403]
 [ 0.01135806]]
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
Accuracy:  1.0
'''


Running...
Step:  0 	 Loss:  1.00032306
W1:  [[0.769314051 0.374498963]
 [-0.0571876429 0.00471873302]] 	 W2:  [[0.342847496]
 [-0.272891641]]
Step:  1000 	 Loss:  0.693004
W1:  [[0.762940407 0.357756436]
 [-0.153855056 -0.0394066907]] 	 W2:  [[0.155675873]
 [-0.897824526]]
Step:  2000 	 Loss:  0.691491425
W1:  [[0.841490209 0.349691421]
 [-0.398999482 -0.108898751]] 	 W2:  [[0.407773465]
 [-0.912501633]]
Step:  3000 	 Loss:  0.675787866
W1:  [[1.2871542 0.505471]
 [-1.15666 -0.272968143]] 	 W2:  [[1.11888373]
 [-1.01361811]]
Step:  4000 	 Loss:  0.489138663
W1:  [[2.99547791 1.73985684]
 [-3.18689704 -1.43698788]] 	 W2:  [[3.30448842]
 [-2.09208155]]
Step:  5000 	 Loss:  0.14952606
W1:  [[4.51025438 3.9628129]
 [-4.6665535 -3.66153097]] 	 W2:  [[5.84639549]
 [-5.01734495]]
Step:  6000 	 Loss:  0.0671663
W1:  [[5.12131834 4.85792398]
 [-5.27038145 -4.56899118]] 	 W2:  [[7.25984192]
 [-6.62003]]
Step:  7000 	 Loss:  0.0414245278
W1:  [[5.44596338 5.30720949]
 [-5.60027838 -5.02377176]] 

'\nHypothesis:  [[ 0.01338224]\n [ 0.98166382]\n [ 0.98809403]\n [ 0.01135806]]\nCorrect:  [[ 0.]\n [ 1.]\n [ 1.]\n [ 0.]]\nAccuracy:  1.0\n'